<a href="https://colab.research.google.com/github/subhamyadav580/SMS-Spam-Classifier-Using-Different-Machine-Learning-Algorithms/blob/main/SMS_Spam_Classifier_of_Using_Different_Machine_Learning_Algorithms_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
%cd '/content/drive/My Drive/'

/content/drive/My Drive


In [81]:
pip install bcolz

In [82]:
pip install isoweek

In [83]:
pip install pandas-summary

In [84]:
import pandas as pd
import numpy as np
from preprocessing.imports import *
from preprocessing.structured import *

In [224]:
loaded_data = pd.read_csv('data/spam.csv', encoding='Windows-1252')

In [225]:
loaded_data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [226]:
loaded_data.drop(['Unnamed: 2', 'Unnamed: 3',	'Unnamed: 4'], inplace=True, axis=1)

In [227]:
loaded_data.rename(columns={'v1':'label', 'v2':'Messages'}, inplace=True)

In [228]:
loaded_data

,label,Messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [229]:
loaded_data.isnull().sum()

label       0
Messages    0
dtype: int64

In [260]:
labels = (pd.get_dummies(loaded_data['label'], drop_first=True)).to_numpy()

In [261]:
labels = labels.flatten() 

In [238]:
messages = loaded_data['Messages'].to_numpy().tolist()

In [92]:
import re

In [93]:
# Replacing Emial id's with the single string using regular expression

messages = [ re.sub(r"[a-zA-Z0-9.!#$%&'*+/=?^_`{|}~-]+@[a-zA-Z]+[.]+[a-zA-Z]+[.]?[a-zA-Z]*", 'EMAILID', message) for message in messages ]

# Replacing web address with the single string using regular expression

messages = [ re.sub(r"https?:\/\/w{0,3}\w*?\.(\w*?\.)?\w{2,3}\S*|www\.(\w*?\.)?\w*?\.\w{2,3}\S*|(\w*?\.)?\w*?\.\w{2,3}[\/\?]\S*", 'WEBADDRESS', message) for message in messages ]

# Replacing Phone number with the single string using regular expression

messages = [ re.sub(r"\d{10}\d{0,9}", 'PHONENUMBER', message) for message in messages ]




In [94]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [95]:
tokenize_sentences = [sent_tokenize(lst) for lst in messages]

In [155]:
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [176]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

In [177]:
def removePunctuations(word):
    text = ""
    for i in word:
        if i not in punctuations:
            text += i
    return stemmer.stem(text)

In [186]:
def wordstokenize(message):
  tokenized_words = []
  for sentence in message:
    sentence = sentence.lower()
    tokenized = word_tokenize(sentence)
    for word in tokenized:
      if word in stop_words:
        word = ""
      if word:
         word = removePunctuations(word)
      if word is not "":
        tokenized_words.append(word)
  return tokenized_words

In [187]:
words_tokenize = [wordstokenize(message) for message in tokenize_sentences]

In [193]:
def flatten(my_list):
  result = []
  for el in my_list:
    if isinstance(el, list):
      flat_list = flatten(el)
      result += flat_list
    else:
      result.append(el)
  return result

In [194]:
list_of_words = flatten(words_tokenize)

In [198]:
def create_features_dictionary(document_tokens):
  features_dictionary = {}
  index = 0
  for token in document_tokens:
    if token not in features_dictionary:
      features_dictionary[token] = index
      index += 1
  return features_dictionary

In [199]:
words_dictionary = create_features_dictionary(list_of_words)

In [201]:
def tokens_to_bow_vector(document_tokens, features_dictionary):
  bow_vector = [0] * len(features_dictionary)
  for token in document_tokens:
    if token in features_dictionary:
      feature_index = features_dictionary[token]
      bow_vector[feature_index] += 1
  return bow_vector

In [246]:
bow_of_vectors = [tokens_to_bow_vector(message, words_dictionary) for message in words_tokenize]

In [270]:
bow_of_vectors = np.asarray(bow_of_vectors)

In [272]:
from sklearn.model_selection import train_test_split

In [273]:
X_train, X_test, y_train, y_test = train_test_split(bow_of_vectors, labels, test_size=0.20, random_state=42)

In [274]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4457, 7533), (1115, 7533), (4457,), (1115,))

In [275]:
from sklearn.linear_model import LogisticRegression

In [276]:
model = LogisticRegression()

In [277]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [278]:
model.predict_proba(X_test)

array([[0.79951, 0.20049],
       [0.96978, 0.03022],
       [0.19552, 0.80448],
       ...,
       [0.97294, 0.02706],
       [0.99219, 0.00781],
       [0.82961, 0.17039]])